In [3]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

# Data Reading

In [4]:
data = pd.read_csv('../data/hits.csv', index_col=0)
data.head()

,EventID,TrackID,PdgCode,DetectorID,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb,Px,...,Z,dist2Wire,Wx1,Wx2,Wy1,Wy2,Wz,Wz1,Wz2,Label
79,2,2.0,-211.0,10002284.0,1,0,0,0,284,0.866347,...,2581.149902,0.042245,-250.000000,250.000000,0.868600,0.868600,2581.1500,2581.1500,2581.1500,0.0
111,2,2.0,-211.0,42012282.0,4,2,0,1,282,0.867240,...,3542.247803,0.478996,-248.589486,249.507863,27.037487,-16.540384,3542.3793,3542.3793,3542.3793,0.0
81,2,2.0,-211.0,11012288.0,1,1,0,1,288,0.866625,...,2592.249023,0.310706,-248.587499,249.509850,-27.060200,16.517671,2592.3793,2592.3793,2592.3793,0.0
43,2,3.0,-13.0,11012286.0,1,1,0,1,286,-0.239448,...,2592.249268,0.117904,-248.894287,249.203062,-23.553595,20.024276,2592.3793,2592.3793,2592.3793,1.0
88,2,2.0,-211.0,20012285.0,2,0,0,1,285,0.866401,...,2782.250977,0.437575,-250.000000,250.000000,-0.011400,-0.011400,2782.2500,2782.2500,2782.2500,0.0


In [5]:
data.columns

Index([u'EventID', u'TrackID', u'PdgCode', u'DetectorID', u'StatNb', u'ViewNb',
       u'PlaneNb', u'LayerNb', u'StrawNb', u'Px', u'Py', u'Pz', u'X', u'Y',
       u'Z', u'dist2Wire', u'Wx1', u'Wx2', u'Wy1', u'Wy2', u'Wz', u'Wz1',
       u'Wz2', u'Label'],
      dtype='object')

# Choosing an event

In [6]:
event = data[data.EventID==2]

# Function Initialization

## distance between 2 skew lines

In [93]:
def cross(vec1, vec2):
    return T.as_tensor([
        vec1[1]*vec2[2] - vec1[2]*vec2[1],
        vec1[2]*vec2[0] - vec1[0]*vec2[2],
        vec1[0]*vec2[1] - vec1[1]*vec2[0]])

In [185]:
#arr = np.array(x1, y1, z1, x2, y2, z2)
def points2vec(arr):
    a0 = np.array([arr[0], arr[1], arr[2]])
    a = np.array([arr[3]-arr[0], arr[4]-arr[1], arr[5]-arr[2]])
    return a0, a

In [95]:
a0 = T.vector("my_vector", dtype='float64')
a = T.vector("my_vector", dtype='float64')
b0 = T.vector("my_vector", dtype='float64')
b = T.vector("my_vector", dtype='float64')
num = T.scalar("n_tubes", dtype='int64')

In [122]:
dist = T.sqrt(T.sum(((a0-b0)*cross(a, b))**2))/T.sqrt(T.sum(cross(a,b)**2))

In [123]:
distance_between_skew_lines = theano.function([a0, a, b0, b], dist)

## retina artifitial response

In [154]:
points = T.matrix("points", dtype='float64')
directions = T.matrix("directions", dtype='float64')
track0 = T.vector("track0", dtype='float64')
track = T.vector("track", dtype='float64')
sigma = T.scalar("sigma", dtype="float64")

In [200]:
rs, updates = theano.scan(fn = lambda point, direction, tr0, tr, s:
                         T.exp(-(T.sqrt(T.sum(((point-tr0)*cross(direction, tr))**2))/T.sqrt(T.sum(cross(direction,tr))**2))**2/s**2),
                         sequences=[points, directions],
                         non_sequences=[track0, track, sigma])

In [201]:
r = rs.sum()
R = theano.function([track0, track, points, directions, sigma], r)

# Test

## test distance between 2 skew lines

In [124]:
track1 = event[(event.TrackID==2)&(event.StatNb<3)]
track2 = event[(event.TrackID==3)&(event.TrackID<3)]
noise = event[(event.TrackID!=2)&(event.TrackID!=3)&(event.TrackID<3)]

In [125]:
x0=track1.X.values[0]
x1=track1.X.values[1]
y0=track1.Y.values[0]
y1=track1.Y.values[1]
z0=track1.Z.values[0]
z1=track1.Z.values[1]

In [191]:
track_point, track_direction = points2vec(np.array([x0, y0, z0, x1, y1, z1]))

In [192]:
params = track1[['Wx1', 'Wy1', 'Wz', 'Wx2', 'Wy2', 'Wz']].values[1]

In [193]:
tube_point, tube_direction = points2vec(params)

In [194]:
distance_between_skew_lines(track_point, track_direction, tube_point, tube_direction)

array(39.49327433542012)

## test retina artifitial response

In [202]:
A0 = []
A = []
for i in range(len(track1.index)):
    a0, a = points2vec(track1[['Wx1', 'Wy1', 'Wz', 'Wx2', 'Wy2', 'Wz']].values[i])
    A0.append(a0)
    A.append(a)
A0 = np.array(A0)
A = np.array(A)

In [205]:
R(track_point, track_direction, A0, A, 2)

array(8.332593368599591)

In [196]:
track1

,EventID,TrackID,PdgCode,DetectorID,StatNb,ViewNb,PlaneNb,LayerNb,StrawNb,Px,...,Z,dist2Wire,Wx1,Wx2,Wy1,Wy2,Wz,Wz1,Wz2,Label
79,2,2.0,-211.0,10002284.0,1,0,0,0,284,0.866347,...,2581.149902,0.042245,-250.000000,250.000000,0.868600,0.868600,2581.1500,2581.1500,2581.1500,0.0
81,2,2.0,-211.0,11012288.0,1,1,0,1,288,0.866625,...,2592.249023,0.310706,-248.587499,249.509850,-27.060200,16.517671,2592.3793,2592.3793,2592.3793,0.0
88,2,2.0,-211.0,20012285.0,2,0,0,1,285,0.866401,...,2782.250977,0.437575,-250.000000,250.000000,-0.011400,-0.011400,2782.2500,2782.2500,2782.2500,0.0
85,2,2.0,-211.0,13002284.0,1,3,0,0,284,0.866653,...,2611.149902,0.097779,-250.000000,250.000000,0.868600,0.868600,2611.1500,2611.1500,2611.1500,0.0
86,2,2.0,-211.0,13102284.0,1,3,1,0,284,0.866610,...,2613.750732,0.337356,-250.000000,250.000000,0.428600,0.428600,2613.7500,2613.7500,2613.7500,0.0
87,2,2.0,-211.0,20002284.0,2,0,0,0,284,0.866590,...,2781.148926,0.439979,-250.000000,250.000000,0.868600,0.868600,2781.1500,2781.1500,2781.1500,0.0
97,2,2.0,-211.0,23102284.0,2,3,1,0,284,0.868264,...,2813.749756,0.074288,-250.000000,250.000000,0.428600,0.428600,2813.7500,2813.7500,2813.7500,0.0
89,2,2.0,-211.0,20102284.0,2,0,1,0,284,0.866252,...,2783.750000,0.005764,-250.000000,250.000000,0.428600,0.428600,2783.7500,2783.7500,2783.7500,0.0
93,2,2.0,-211.0,22012281.0,2,2,0,1,281,0.866556,...,2802.250000,0.034292,-248.436092,249.661257,28.790790,-14.787082,2802.3793,2802.3793,2802.3793,0.0
82,2,2.0,-211.0,11112288.0,1,1,1,1,288,0.866595,...,2594.850342,0.121902,-248.549150,249.548199,-27.498526,16.079345,2594.9793,2594.9793,2594.9793,0.0


# 2D projections

In [207]:
size = 10
grid = np.ndarray(shape=(size, size, size, size), dtype='float64')

In [ ]:
for i in np.linspace(0, 0.2, size):
    for j in np.linspace(0, 400, size):
        for m in np.linspace(0, 0.1, size):
            

In [208]:
range(0, 0.2, 0.1)

TypeError: range() integer end argument expected, got float.